### Simple ModelGrid Demo

This demo shows how the ModelGrid classes can work, where all model grid functionality is in the model grid class.  Only the geographic reference information and related methods remain in SpatialReference.

In [ ]:
import os
import sys
import numpy as np
# run installed version of flopy or add local path
try:
    from flopy.discretization.structuredgrid import StructuredGrid
    from flopy.utils.reference import SpatialReference
except:
    fpth = os.path.abspath(os.path.join('..', '..'))
    sys.path.append(fpth)
    from flopy.discretization.structuredgrid import StructuredGrid
    from flopy.utils.reference import SpatialReference

import matplotlib as mpl
import matplotlib.pyplot as plt
import flopy

print(sys.version)
print('numpy version: {}'.format(np.__version__))
print('matplotlib version: {}'.format(mpl.__version__))
print('flopy version: {}'.format(flopy.__version__))

Technically you need to create both a SpatialReference object and a ModelGrid object, but in practice the code looks very similar and can easily be implemented in one line.

In [ ]:
mg = StructuredGrid(delc=np.ones(10), delr=np.ones(10), xoff=10, yoff=10, angrot=20, 
                    epsg=26715, lenuni=1)

### Transform, xcenters, xedges

Transform, xcenters, xedges behaves the same but is now a method of the ModelGrid, but has the same functionatily. 

In [ ]:
mg.get_coords(10, 10)

In [ ]:
mg.get_local_coords(11.82170571129881, 13.906660505012088)

Methods to get cell centers of cells from the model grid are now in the ModelGrid classes.

In [ ]:
mg.xcellcenters[1, 7] # x-coordinate at row 1, column 7

In [ ]:
mg.xcellcenters[[1, 3], [7, 7]]

In [ ]:
plt.scatter(mg.xcellcenters.ravel(), mg.ycellcenters.ravel())
plt.scatter(mg.xyzvertices[0].ravel(), mg.xyzvertices[1].ravel(), s=10)

### Model Coordinates

The model grid outputs everything in model coordinates if reference information is not given.

In [ ]:
mg.set_coord_info()
plt.scatter(mg.xcellcenters.ravel(), mg.ycellcenters.ravel())
plt.scatter(mg.xyzvertices[0].ravel(), mg.xyzvertices[1].ravel(), s=10)

### Grid lines

In [ ]:
def plot_lines(lines):
    fig, ax = plt.subplots()
    for ln in lines:
        plt.plot([ln[0][0], ln[1][0]], [ln[0][1], ln[1][1]])

In [ ]:
plot_lines(mg.grid_lines)
plt.ylim(11, -1)

In [ ]:
mg.set_coord_info(xoff=10, yoff=10, angrot=20, epsg=26715)
plot_lines(mg.grid_lines)
plt.scatter(11.9, 13.5, c='r')

### Vertices

In [ ]:
mg.set_coord_info()
mg.xyzvertices[0]

In [ ]:
mg.set_coord_info(xoff=10, yoff=10, angrot=20, epsg=26715)
mg.xyzvertices[0]

### Getting the Model Grid from the Model

In [ ]:
ml = flopy.modflow.Modflow(modelname='new_model', exe_name='mf2005', 
                           version='mf2005', model_ws=os.path.join('data'))

dis = flopy.modflow.ModflowDis(ml,nlay=5,nrow=5,ncol=5,delr=[50,50,50,50,50],
                               delc=[50,50,50,50,50],top=0.0,botm=-50.0,laycbd=0) 

ibound = np.ones((5, 5, 5), dtype=np.int)
start = np.zeros((5, 5, 5), dtype=np.float)
bas = flopy.modflow.ModflowBas(ml,ibound=ibound,strt=start)

Use the .modelgrid attribute to get the model grid from the model. 

In [ ]:
ml.modelgrid.xyzvertices[0]

By default the flopy model class caches the model grid. If you later modify model discritization information the model grid can end up out of sync with the model discritization information.

In [ ]:
dis.delr=[10,10,10,10,10]
ml.modelgrid.xyzvertices[0]

To resync the model grid with the latest model discritization information call update_modelgrid. Calling the update_modelgrid method clears the cached model grid and forces flopy to create a new model grid object on the next call to modelgrid. The current model discritization information is used to create the new model grid.

In [ ]:
ml.update_modelgrid()
ml.modelgrid.xyzvertices[0]